# Submit format

      spark-submit \
      --class <main-class> \
      --master <master-url> \
      --deploy-mode <deploy-mode> \
      --conf <key>=<value> \
      ... # other options
      <application-jar> or <apllication.py> \
      [application-arguments]

In [2]:
# Let's submit a simple Spark application that read data and print 20 rows to screen.

In [ ]:
! hdfs dfs -put /home/train/datasets/market5mil_snappyparquet/ /user/train/datasets

# Simple submit

    (venvspark) [train@localhost 05_submit_spark_applications]$ spark-submit \
    --master yarn \
    --deploy-mode client \
    simple_submit.py \
    -i file:///user/train/datasets/market5mil_snappyparquet \
    -f parquet \
    -c snappy

In [1]:
! ls -l

total 12
-rw-rw-r--. 1 train train 7043 Nov 12 21:18 01_spark-submit.ipynb
-rw-rw-r--. 1 train train 1818 Nov  5 19:31 simple_submit.py


# Submit with configuration

    (venvspark) [train@localhost 05_submit_spark_applications]$ spark-submit \
        --master yarn \
        --deploy-mode client \
        --num-executors 2 \
        --conf spark.sql.autoBroadcastJoinThreshold=-1 \
        simple_submit.py \
        -i file:///home/train/datasets/market5mil_snappyparquet \
        -f parquet \
        -c snappy \
        --appName SparkApplicationSubmit

# Submit with extra packages

    (venvspark) [train@localhost 05_submit_spark_applications]$ spark-submit \
        --master yarn \
        --deploy-mode client \
        --num-executors 2 \
        --packages "org.apache.spark:spark-avro_2.12:3.0.0" \
        --conf spark.sql.autoBroadcastJoinThreshold=-1 \
        most_selling_products_to_postgresql.py \
        -i hdfs://localhost:9000/user/train/datasets/market5mil_snappyparquet \
        -f parquet \
        -c snappy \
        --appName SparkApplicationSubmit

In [6]:
# After submit you see the following logs

    :: resolving dependencies :: org.apache.spark#spark-submit-parent-630b8af2-4eaf-4f64-a2de-15b84345333e;1.0
            confs: [default]
            found org.apache.spark#spark-avro_2.12;3.0.0 in central
            found org.spark-project.spark#unused;1.0.0 in central
    :: resolution report :: resolve 950ms :: artifacts dl 22ms
            :: modules in use:
            org.apache.spark#spark-avro_2.12;3.0.0 from central in [default]
            org.spark-project.spark#unused;1.0.0 from central in [default]
            ---------------------------------------------------------------------
            |                  |            modules            ||   artifacts   |
            |       conf       | number| search|dwnlded|evicted|| number|dwnlded|
            ---------------------------------------------------------------------
            |      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
            ---------------------------------------------------------------------
    :: retrieving :: org.apache.spark#spark-submit-parent-630b8af2-4eaf-4f64-a2de-15b84345333e
            confs: [default]


# Submit with extra jars

    (venvspark) [train@localhost 05_submit_spark_applications]$ spark-submit \
            --master yarn \
            --deploy-mode client \
            --num-executors 2 \
            --jars "hdfs://localhost:9000/tmp/extra-jars/org.apache.spark_spark-avro_2.12-3.0.0.jar" \
            --conf spark.sql.autoBroadcastJoinThreshold=-1 \
            most_selling_products_to_postgresql.py \
            -i hdfs://localhost:9000/user/train/datasets/market5mil_snappyparquet \
            -f parquet \
            -c snappy \
            --appName SparkApplicationSubmit

--jars.

Spark uses the following URL scheme to allow different strategies for disseminating jars:

<strong>file:</strong> - Absolute paths and file:/ URIs are served by the driver’s HTTP file server, and every executor pulls the file from the driver HTTP server.<br>
<strong>hdfs:</strong>, http:, https:, ftp: - these pull down files and JARs from the URI as expected <br>
<strong>local:</strong> - a URI starting with local:/ is expected to exist as a local file on each worker node. This means that no network IO will be incurred, and works well for large files/JARs that are pushed to each worker, or shared via NFS, GlusterFS, etc.